In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation


In [9]:
import pandas as pd
from surprise import Dataset
from surprise import Reader

from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split as train_test_split_surprise
from surprise import Dataset, SVD
from surprise.model_selection import cross_validate, GridSearchCV
from sklearn.model_selection import train_test_split

import numpy as np

### Read Provided Train Test validation

In [3]:
train_set = pd.read_csv("../../data/recommendation/interactions_train.csv")
test_set = pd.read_csv("../../data/recommendation/interactions_test.csv")
validation_set = pd.read_csv("../../data/recommendation/interactions_validation.csv")
train_set.shape, test_set.shape, validation_set.shape

((698901, 6), (12455, 6), (7023, 6))

In [4]:
698901+12455+7023

718379

In [6]:
raw_interactions = pd.read_csv("../../data/recommendation/RAW_interactions.csv")
raw_interactions.shape

(1132367, 5)

### Create Train/Test/validation from raw

In [13]:
TRAIN__Validation_SIZE = 0.8
TEST_SIZE = 0.2

In [42]:
train_validation, test_set = train_test_split(raw_interactions, test_size=TEST_SIZE)
train_validation.shape, test_set.shape

((905893, 5), (226474, 5))

In [43]:
905893 + 226474

1132367

### Train SVD matrix Factorisation

In [50]:
reader = Reader(rating_scale=(1, 5))
cv_data = Dataset.load_from_df(train_validation[["user_id", "recipe_id", "rating"\
                                                                           ]], reader)

In [58]:
param_grid = {"n_factors":[20, 50] ,"n_epochs": [10, 15], "lr_all": [0.002, 0.005]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5, n_jobs = -2)

gs.fit(cv_data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

print(gs.best_params["mae"])

1.218094140876164
{'n_factors': 20, 'n_epochs': 15, 'lr_all': 0.005}
{'n_factors': 20, 'n_epochs': 15, 'lr_all': 0.005}


In [59]:
cv_results_df = pd.DataFrame.from_dict(gs.cv_results)
cv_results_df

,split0_test_rmse,split1_test_rmse,split2_test_rmse,split3_test_rmse,split4_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,...,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_factors,param_n_epochs,param_lr_all
0,1.228697,1.222297,1.223658,1.224391,1.227176,1.225244,0.002349,7,0.753424,0.750137,...,0.001077,7,3.104795,0.285593,2.142331,0.043642,"{'n_factors': 20, 'n_epochs': 10, 'lr_all': 0....",20,10,0.002
1,1.222545,1.216202,1.217771,1.218116,1.220700,1.219067,0.002261,2,0.745213,0.742198,...,0.001041,3,3.418055,0.057300,2.247503,0.112610,"{'n_factors': 20, 'n_epochs': 10, 'lr_all': 0....",20,10,0.005
2,1.225618,1.219103,1.220609,1.221104,1.223918,1.222070,0.002361,5,0.748299,0.744931,...,0.001136,5,5.038678,0.246600,2.584927,0.124022,"{'n_factors': 20, 'n_epochs': 15, 'lr_all': 0....",20,15,0.002
3,1.221387,1.215518,1.216680,1.217148,1.219737,1.218094,0.002148,1,0.742132,0.739437,...,0.000949,1,5.407024,0.045598,2.592297,0.112011,"{'n_factors': 20, 'n_epochs': 15, 'lr_all': 0....",20,15,0.005
4,1.229484,1.223111,1.224706,1.225294,1.227844,1.226088,0.002281,8,0.753773,0.750606,...,0.001036,8,5.595094,0.050464,2.665056,0.189076,"{'n_factors': 50, 'n_epochs': 10, 'lr_all': 0....",50,10,0.002
5,1.223622,1.217173,1.219072,1.219182,1.221810,1.220172,0.002270,4,0.745916,0.742958,...,0.001079,4,5.840886,0.037470,2.945850,0.388703,"{'n_factors': 50, 'n_epochs': 10, 'lr_all': 0....",50,10,0.005
6,1.226369,1.220202,1.221285,1.221693,1.224812,1.222872,0.002325,6,0.748669,0.745687,...,0.001095,6,9.108575,0.297556,2.598949,0.253296,"{'n_factors': 50, 'n_epochs': 15, 'lr_all': 0....",50,15,0.002
7,1.222967,1.216686,1.218197,1.218461,1.220920,1.219446,0.002223,3,0.743177,0.740046,...,0.001130,2,7.053559,0.782029,1.927421,0.273523,"{'n_factors': 50, 'n_epochs': 15, 'lr_all': 0....",50,15,0.005


### Test RMSE

In [60]:
#Build model on full cv set
algo = gs.best_estimator["rmse"]
algo.fit(cv_data.build_full_trainset())

In [67]:
test_set_surprise = Dataset.load_from_df(test_set[["user_id", "recipe_id", "rating"]], reader)
_, test_set_surprise = train_test_split_surprise(test_set_surprise, test_size=1.0)

In [68]:
len(test_set_surprise)

226474

In [69]:
predictions = algo.test(test_set_surprise)
accuracy.rmse(predictions, verbose=True)

RMSE: 1.2117


1.2117454568213044

### Analyse with Metadata

In [ ]:
recipe_metadata = 

train_validation, test_set = train_test_split_surprise(full_data, test_size=0.15)

train_validation.n_ratings, len(test_set), train_validation.n_ratings + len(test_set)

train_validation.n_items, train_validation.n_users

### Build SVD Matrix Factorisation

svd = SVD(verbose=True, n_epochs=10)
cross_validate(svd, train_validation, measures=['RMSE', 'MAE'], cv=10, verbose=True)

dir(train_validation)